# Script for creating dataset with game matchups and sentiment for next game
| GAME_ID | TEAM_ID | NEXT_GAME_ID | SENTIMENT |
|---------|---------|--------------|-----------|
| ...     | ...     | ...          | ...       |

    

In [1]:
import os
from typing import List

from utils import *

In [2]:
filenames: List[str] = os.listdir("resources/comments")
gameIDs = [int(filename.removesuffix(".tsv")) for filename in filenames]

f"Posts: {len(filenames)}"

'Posts: 419'

In [3]:
ready_to_go = 0
for filename in filenames:
    post = pd.read_csv("resources/comments/" + filename, sep="\t")
    classified = 1 if len(set(post["TEAM_ABBREVIATION"].tolist())) > 1 else 0
    if classified == 0:
        print(filename)
        
    ready_to_go += classified
    
f"Ready to go posts: {ready_to_go}"

'Ready to go posts: 419'

In [4]:
season_23_24 = pd.read_csv(f"resources/game-logs/season_2023-24.tsv", sep='\t')
season_23_24

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612737,22301188,"APR 14, 2024",ATL @ IND,L,36,46,0.439,240,39,...,0.926,9,23,32,25,6,5,15,12,115
1,1610612737,22301178,"APR 12, 2024",ATL @ MIN,L,36,45,0.444,240,40,...,0.680,9,31,40,23,4,1,14,25,106
2,1610612737,22301159,"APR 10, 2024",ATL vs. CHA,L,36,44,0.450,240,43,...,0.625,7,31,38,35,7,2,16,20,114
3,1610612737,22301147,"APR 09, 2024",ATL vs. MIA,L,36,43,0.456,290,45,...,0.786,17,42,59,28,13,2,15,23,111
4,1610612737,22301130,"APR 06, 2024",ATL @ DEN,L,36,42,0.462,240,37,...,0.828,11,27,38,30,8,2,16,18,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,1610612766,22300133,"NOV 04, 2023",CHA @ IND,W,2,3,0.400,240,47,...,1.000,12,29,41,23,5,10,13,20,125
2456,1610612766,22300122,"NOV 01, 2023",CHA @ HOU,L,1,3,0.250,240,45,...,0.720,15,29,44,21,9,8,16,19,119
2457,1610612766,22300101,"OCT 30, 2023",CHA vs. BKN,L,1,2,0.333,240,47,...,0.950,8,31,39,33,8,5,9,23,121
2458,1610612766,22300077,"OCT 27, 2023",CHA vs. DET,L,1,1,0.500,240,33,...,0.765,11,30,41,20,13,7,18,23,99


In [5]:
nextGames = {
    gameID : findNextGameID(gameID, season_23_24) for gameID in gameIDs
}

In [27]:
has_next_game = 0
next_games_set = set()
for key, value in nextGames.items():
    for teamId, nextGameID in value.items():
        has_next_game += 1 if nextGameID and not nextGameID else 0
        if nextGameID != "":
            next_games_set.add(nextGameID)

len(next_games_set)

484

In [7]:
comments = 0
comments_all = 0
for filename in filenames:
    post = pd.read_csv("resources/comments/" + filename, sep="\t")
    comments += len(post[post["TEAM_ABBREVIATION"].notna()].values)
    comments_all += len(post.values)
        

f"Number of all comments: {comments_all}, number of valid comments: {comments}, average number of comments per post: {comments_all / len(filenames)}, average number of valid comments per post: {comments / len(filenames)}"

'Number of all comments: 26910, number of valid comments: 15534, average number of comments per post: 64.22434367541766, average number of valid comments per post: 37.07398568019093'

In [8]:
nextGames

{22300567: {1610612756: 22300588, 1610612758: 22300583},
 22300658: {1610612739: 22300675, 1610612746: 22300676},
 22300659: {1610612738: 22300670, 1610612740: 22300679},
 22300677: {1610612751: 22300698, 1610612756: 22300690},
 22300680: {1610612742: 22300702, 1610612750: 22300694},
 22300681: {1610612753: 22300694, 1610612759: 22300696},
 22300686: {1610612755: 22300698, 1610612762: 22300711},
 22300687: {1610612739: 22300703, 1610612763: 22300693},
 22300688: {1610612746: 22300708, 1610612765: 22300704},
 22300689: {1610612748: 22300708, 1610612764: 22300705},
 22300690: {1610612737: 22300699, 1610612756: 22300705},
 22300692: {1610612745: 22300709, 1610612761: 22300710},
 22300694: {1610612750: 22300709, 1610612753: 22300704},
 22300695: {1610612760: 22300710, 1610612766: 22300707},
 22300696: {1610612740: 22300718, 1610612759: 22300703},
 22300698: {1610612751: 22300717, 1610612755: 22300715},
 22300700: {1610612741: 22300723, 1610612758: 22300714},
 22300702: {1610612742: 2230071

In [28]:
df = DataFrame()

for key, value in nextGames.items():
    gameID = key
    for teamId, nextGameID in value.items():
        row = {
            "GAME_ID": [gameID],
            "TEAM_ID": [teamId],
            "NEXT_GAME_ID": [nextGameID],
        }
        df = pd.concat([df, DataFrame(row)], ignore_index=True)
df

,GAME_ID,TEAM_ID,NEXT_GAME_ID
0,22300567,1610612756,22300588
1,22300567,1610612758,22300583
2,22300658,1610612739,22300675
3,22300658,1610612746,22300676
4,22300659,1610612738,22300670
...,...,...,...
833,22301198,1610612762,
834,22301199,1610612745,
835,22301199,1610612746,
836,22301200,1610612757,


In [18]:
# len(set(df.dropna().astype({"NEXT_GAME_ID": int})["NEXT_GAME_ID"].tolist()))
len(set(df[df["NEXT_GAME_ID"].notna()]["NEXT_GAME_ID"].tolist()))

485

In [22]:
next_games = season_23_24[season_23_24["Game_ID"].isin(next_games_set)][["Game_ID", "Team_ID"]].copy()
next_games.rename(columns={"Game_ID": "NEXT_GAME_ID", "Team_ID": "TEAM_ID"}, inplace=True)
next_games.head()

,NEXT_GAME_ID,TEAM_ID
0,22301188,1610612737
1,22301178,1610612737
2,22301159,1610612737
3,22301147,1610612737
4,22301130,1610612737


In [23]:
current_next_games = df.copy().dropna().astype({"NEXT_GAME_ID": str})
current_next_games.head()

,GAME_ID,TEAM_ID,NEXT_GAME_ID
0,22300567,1610612756,22300588
1,22300567,1610612758,22300583
2,22300658,1610612739,22300675
3,22300658,1610612746,22300676
4,22300659,1610612738,22300670


In [29]:
all_next_games = next_games.astype({"NEXT_GAME_ID": str}).merge(current_next_games, on=["NEXT_GAME_ID", "TEAM_ID"], how="left")
all_next_games.head()

,NEXT_GAME_ID,TEAM_ID,GAME_ID
0,22301188,1610612737,22301178.0
1,22301178,1610612737,22301159.0
2,22301159,1610612737,22301147.0
3,22301147,1610612737,22301130.0
4,22301130,1610612737,22301124.0


In [32]:
sentiment = pd.read_csv(f"resources/sentiment-analysis/fans-sentiment.tsv", sep="\t")

sentiment.head()

,GAME_ID,TEAM_ID,SENTIMENT
0,22300567,1610612756,0.639242
1,22300567,1610612758,0.295156
2,22300658,1610612739,0.626677
3,22300658,1610612746,0.408076
4,22300659,1610612738,0.553635


In [39]:
all_next_games = all_next_games
all_next_games["GAME_ID"] = all_next_games["GAME_ID"].map(lambda x: int(x) if not math.isnan(x) else -1)

next_current_sentiment = all_next_games.merge(sentiment, on=["GAME_ID", "TEAM_ID"], how="left")
next_current_sentiment["SENTIMENT"] = next_current_sentiment["SENTIMENT"].fillna(0).astype(float)


next_current_sentiment.head(10)

,NEXT_GAME_ID,TEAM_ID,GAME_ID,SENTIMENT
0,22301188,1610612737,22301178,0.408076
1,22301178,1610612737,22301159,0.408076
2,22301159,1610612737,22301147,0.289532
3,22301147,1610612737,22301130,0.459519
4,22301130,1610612737,22301124,0.408076
5,22301124,1610612737,22301104,0.553862
6,22301104,1610612737,22301091,0.539655
7,22301091,1610612737,22301076,0.499100
8,22301076,1610612737,-1,0.000000
9,22301060,1610612737,22301051,0.420261


In [42]:
next_current_sentiment[["GAME_ID", "TEAM_ID", "NEXT_GAME_ID", "SENTIMENT"]].to_csv("resources/sentiment-analysis/next-game-sentiment.tsv", sep="\t", index=False)